In [ ]:
def install_umep(MODELS_FOLDER, MODEL_SOURCE_FOLDER):

    import os
    import sys
    import geopandas
    import matplotlib
    import momepy
    import numpy
    import pandas
    import pvlib
    import pyepw
    import pytz
    import rasterio
    import scipy
    import shapely
    import tqdm
    import pythermalcomfort
    import pyproj
    import xarray
    import rioxarray
    import toml

    # INFO: https://github.com/UMEP-dev/umep-core

    # Go to models folder
    os.chdir(MODELS_FOLDER)
    !pwd

    # Only execute if folder does not exist yet
    if not os.path.exists(MODEL_SOURCE_FOLDER):

        print("Installing UMEP-SOLWEIG ...")

        # Clone repo
        !git clone https://github.com/UMEP-dev/umep-core.git
        os.chdir(MODEL_SOURCE_FOLDER)

        # Now install UMEP
        !pip install . --no-deps
        sys.path.append(MODEL_SOURCE_FOLDER)
        import umep

    else:
        print("> UMEP-SOLWEIG already installed!")
        sys.path.append(MODEL_SOURCE_FOLDER)
        import umep


    # Load the required libraries
    print("> Loading the UMEP-SOLWEIG algorithms ...")
    from umep import (
        skyviewfactor_algorithm,
        wall_heightaspect_algorithm,
    )
    from umep.functions.SOLWEIGpython import Solweig_run


In [ ]:
def get_city_info(city):
    if city == "Timbuktu":
      cityInfo = {
          "day_of_interest": "2013-05-26",
          "utc_offset": 0,
          "leaf_on_trees_full_year": 0,
          "hours_of_interest": range(7, 20, 1),
      }
    elif city == "Davao":
      cityInfo = {
          "day_of_interest": "2018-04-30",
          "utc_offset": 8,
          "leaf_on_trees_full_year": 1,
          "hours_of_interest": range(6, 19, 1),
          }
    return cityInfo


In [ ]:
def make_interactive_map_roi(MODEL_INPUT_FOLDER):

    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import io
    import base64
    import folium
    from PIL import Image
    import folium
    import geopandas as gpd
    import rioxarray as rxr

    # Read the data
    # Get the list of geojson files in the input folder
    gdf_hex = gpd.read_file(os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "school_hex_utm.geojson"))
    gdf_bbox = gpd.read_file(os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "school_bbox_utm.geojson"))

    # Read the rasters
    ds_dem = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "dem.tif")).squeeze()
    ds_dsm = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "dsm.tif")).squeeze()
    ds_cdsm = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "cdsm.tif")).squeeze()
    ds_lc = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "lc.tif")).squeeze()

    # Reproject the files
    target_crs = "EPSG:4326"
    gdf_hex = gdf_hex.to_crs(target_crs)
    gdf_bbox = gdf_bbox.to_crs(target_crs)
    ds_dsm = ds_dsm.rio.reproject(target_crs)
    ds_dem = ds_dem.rio.reproject(target_crs)
    ds_dsm = ds_dsm-ds_dem
    ds_cdsm = ds_cdsm.rio.reproject(target_crs)
    ds_lc = ds_lc.rio.reproject(target_crs)

    # Helper function to plot raster

    def raster_overlay(data, bounds, colormap, opacity=0.8, mask_zero=True):
        """
        Create a folium raster overlay from a 2D numpy array.

        Parameters:
            data (np.ndarray): 2D array
            bounds (list): [[south, west], [north, east]]
            colormap (str or dict): matplotlib colormap name or dict {val: color}
            opacity (float): layer transparency
            mask_zero (bool): whether to make zero pixels transparent

        Returns:
            folium.raster_layers.ImageOverlay
        """
        # Mask zero values
        if mask_zero:
            data = np.ma.masked_equal(data, 0)

        # Generate image
        if isinstance(colormap, dict):
            # Handle categorical colormap
            normed = np.zeros((*data.shape, 4), dtype=np.uint8)
            for val, hex_color in colormap.items():
                rgba = np.array(mcolors.to_rgba(hex_color)) * 255
                normed[np.where(data == val)] = rgba.astype(np.uint8)
        else:
            cmap = plt.get_cmap(colormap)
            norm = mcolors.Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
            mapped = cmap(norm(data.filled(np.nan)))  # convert masked to nan
            normed = (mapped * 255).astype(np.uint8)

        # Convert to PNG image in memory
        image = Image.fromarray(normed, mode='RGBA')
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        encoded = base64.b64encode(buffer.getvalue()).decode('utf-8')
        image_url = f"data:image/png;base64,{encoded}"

        # Create folium overlay
        return folium.raster_layers.ImageOverlay(
            image_url,
            bounds=bounds,
            opacity=opacity,
            interactive=True,
            cross_origin=False
        )

    # Create a map centered around the first geometry (assuming at least one file exists)
    centroid = gdf_hex.geometry.union_all().centroid
    m = folium.Map(
        location=[centroid.y, centroid.x],
        zoom_start=17,
        # tiles='Esri_WorldImagery',
        tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr='Google',
        name='Google Satellite',
        width='70%',
        height='800px'
    )

    # Add each geojson layer to the map
    folium.GeoJson(
        gdf_hex,
        name="School hex",
        style_function=lambda x: {'fillColor': 'none', 'color': 'blue', 'weight': 2}
    ).add_to(m)
    # Add each geojson layer to the map
    folium.GeoJson(
        gdf_bbox,
        name="School box",
        style_function=lambda x: {'fillColor': 'none', 'color': 'yellow', 'weight': 2}
    ).add_to(m)

    # Land cover colormap (value: color)
    lc_colormap = {
        1: '#bcbcbc',  # Paved
        2: '#7b7b7b',  # Buildings
        3: '#71915e',  # Evergreen trees
        4: '#5ca534',  # Deciduous trees
        5: '#48e360',  # Grass
        6: '#8f8c30',  # Bare soil
        7: '#0068d8',  # Water
    }

    # Assume raster data is projected in EPSG:4326
    dsm_data = ds_dsm.squeeze().values
    cdsm_data = ds_cdsm.squeeze().values
    lc_data = ds_lc.squeeze().values

    # Get bounds (south, west, north, east)
    bounds = ds_dsm.rio.bounds()  # same for all if aligned
    folium_bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]

    # Add raster overlays
    # Land Cover
    fg_lc = folium.FeatureGroup(name="Land Cover", show=False)
    lc_overlay = raster_overlay(lc_data, folium_bounds, lc_colormap, mask_zero=False)
    lc_overlay.add_to(fg_lc)
    fg_lc.add_to(m)

    # DSM
    fg_dsm = folium.FeatureGroup(name="DSM", show=False)
    dsm_overlay = raster_overlay(dsm_data, folium_bounds, 'magma', mask_zero=True)
    dsm_overlay.add_to(fg_dsm)
    fg_dsm.add_to(m)

    # CDSM
    fg_cdsm = folium.FeatureGroup(name="CDSM", show=False)
    cdsm_overlay = raster_overlay(cdsm_data, folium_bounds, 'Greens', mask_zero=True)
    cdsm_overlay.add_to(fg_cdsm)
    fg_cdsm.add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    return m


In [ ]:
def make_static_map_roi(MODEL_INPUT_FOLDER, figname):

    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import io
    import base64
    import folium
    from PIL import Image
    import folium
    import geopandas as gpd
    import rioxarray as rxr

    # Read the data
    # Get the list of geojson files in the input folder
    gdf_hex = gpd.read_file(os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "school_hex_utm.geojson"))
    gdf_bbox = gpd.read_file(os.path.join(os.path.dirname(MODEL_INPUT_FOLDER), "school_bbox_utm.geojson"))

    # Read the rasters
    ds_dem = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "dem.tif")).squeeze()
    ds_dsm = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "dsm.tif")).squeeze()
    ds_cdsm = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "cdsm.tif")).squeeze()
    ds_lc = rxr.open_rasterio(os.path.join(MODEL_INPUT_FOLDER, "lc.tif")).squeeze()

    # Reproject the files
    target_crs = "EPSG:4326"
    gdf_hex = gdf_hex.to_crs(target_crs)
    gdf_bbox = gdf_bbox.to_crs(target_crs)
    ds_dsm = ds_dsm.rio.reproject(target_crs)
    ds_dem = ds_dem.rio.reproject(target_crs)
    ds_dsm = ds_dsm-ds_dem
    ds_cdsm = ds_cdsm.rio.reproject(target_crs)
    ds_lc = ds_lc.rio.reproject(target_crs)

    # Ensure data is 2D (remove bands if needed)
    dsm_data = ds_dsm.squeeze().values
    cdsm_data = ds_cdsm.squeeze().values
    lc_data = ds_lc.squeeze().values

    # Optional: mask zeros if they represent nodata
    dsm_masked = np.ma.masked_equal(dsm_data, 0)
    cdsm_masked = np.ma.masked_equal(cdsm_data, 0)

    # Land cover color map
    lc_cmap = mcolors.ListedColormap([
        '#bcbcbc',  # 1: Paved
        '#7b7b7b',  # 2: Buildings
        '#71915e',  # 3: Evergreen
        '#5ca534',  # 4: Deciduous
        '#48e360',  # 5: Grass
        '#8f8c30',  # 6: Bare soil
        '#0068d8',  # 7: Water
    ])
    lc_bounds = np.arange(0.5, 8.5, 1)
    lc_norm = mcolors.BoundaryNorm(lc_bounds, lc_cmap.N)

    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(15, 10), constrained_layout=True)

    # DSM and CDSM
    im1 = axes[0].imshow(dsm_masked, cmap='magma')
    axes[0].set_title("DSM and CDSM (m)")
    plt.colorbar(im1, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)

    # # CDSM
    im2 = axes[0].imshow(cdsm_masked, cmap='Greens')
    # axes[1].set_title("CDSM (m)")
    plt.colorbar(im2, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)

    # Land Cover
    im3 = axes[1].imshow(lc_data, cmap=lc_cmap, norm=lc_norm)
    axes[1].set_title("Land Cover")
    cbar = plt.colorbar(im3, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)
    cbar.set_ticks(range(1, 8))
    cbar.set_ticklabels([
        'Paved', 'Buildings', 'Evergreen', 'Deciduous',
        'Grass', 'Bare soil', 'Water'
    ])

    # Remove axis ticks for all
    for ax in axes:
        ax.axis('off')

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def plot_heatwave_characteristics(MODEL_INPUT_FOLDER, figname):

  import os
  import pandas as pd
  import matplotlib.pyplot as plt

  met_file = os.path.join(MODEL_INPUT_FOLDER, "solweig_forcing.txt")

  # Read the file using whitespace delimiter and header in first row
  df = pd.read_csv(met_file, sep=' ')

  # Convert relevant columns to numeric
  df['iy'] = pd.to_numeric(df['iy'], errors='coerce')  # year
  df['id'] = pd.to_numeric(df['id'], errors='coerce')  # day of year
  df['it'] = pd.to_numeric(df['it'], errors='coerce')  # hour
  df['Tair'] = pd.to_numeric(df['Tair'], errors='coerce')  # air temperature
  df['RH'] = pd.to_numeric(df['RH'], errors='coerce')  # relative humidity
  df['U'] = pd.to_numeric(df['U'], errors='coerce')  # wind speed

  # Create datetime index from year (iy), day of year (id), and hour (it)
  df['datetime'] = pd.to_datetime(df['iy'].astype(int).astype(str), format='%Y') + \
                          pd.to_timedelta(df['id'] - 1, unit='D') + \
                          pd.to_timedelta(df['it'], unit='h')

  # Set datetime as index
  df.set_index('datetime', inplace=True)

  # Select all records from the period of interest
  # df_selection = df.loc[START_DATE:END_DATE]

  # plot
  fig, axes = plt.subplots(5, 1, figsize=(12, 10), sharex=True)

  # Plot Kdown
  ax_i = 0
  axes[ax_i].plot(df.index, df['kdown'], color='#FFBF00', label='Kdown')
  axes[ax_i].set_ylabel("SW Down (W/m²)")
  axes[ax_i].grid(True)
  axes[ax_i].legend()
  # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

  # Plot Tair
  ax_i = 1
  axes[ax_i].plot(df.index, df['Tair'], color='firebrick', label='Tair')
  axes[ax_i].set_ylabel("Temperature (°C)")
  axes[ax_i].grid(True)
  axes[ax_i].legend()
  # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

  # Plot RH
  ax_i = 2
  axes[ax_i].plot(df.index, df['RH'], color='navy', label='RH')
  axes[ax_i].set_ylabel("Relative Humidity (%)")
  axes[ax_i].grid(True)
  axes[ax_i].legend()
  # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

  # Plot U
  ax_i = 3
  axes[ax_i].plot(df.index, df['U'], color='olive', label='U')
  axes[ax_i].set_ylabel("Wind Speed (m/s)")
  axes[ax_i].set_xlabel("Time")
  axes[ax_i].grid(True)
  axes[ax_i].legend()
  # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

  # Plot wdir
  ax_i = 4
  axes[ax_i].plot(df.index, df['wdir'], color='0.2', label='WDir')
  axes[ax_i].set_ylabel("Wind Direction (° from N)")
  axes[ax_i].set_xlabel("Time")
  axes[ax_i].grid(True)
  axes[ax_i].legend()
  # axes[ax_i].axvspan(df_selection.index[0], df_selection.index[-1], color='orange', alpha=0.2)

  plt.tight_layout()

  plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
  print("File saved:", os.path.exists(figname))

  plt.show()


In [ ]:
def get_meteo_forcing_day_of_interest(MODEL_INPUT_FOLDER, day):

    import os
    import pandas as pd

    # Read the file using whitespace delimiter and header in first row
    met_file = os.path.join(MODEL_INPUT_FOLDER, "solweig_forcing.txt")
    met_file_selection = met_file.replace('.txt', f'_{day}.txt')

    # Read the file using whitespace delimiter and header in first row
    df = pd.read_csv(met_file, sep=' ')

    # Convert relevant columns to numeric
    df['iy'] = pd.to_numeric(df['iy'], errors='coerce')  # year
    df['id'] = pd.to_numeric(df['id'], errors='coerce')  # day of year
    df['it'] = pd.to_numeric(df['it'], errors='coerce')  # hour
    df['Tair'] = pd.to_numeric(df['Tair'], errors='coerce')  # air temperature
    df['RH'] = pd.to_numeric(df['RH'], errors='coerce')  # relative humidity
    df['U'] = pd.to_numeric(df['U'], errors='coerce')  # wind speed

    # Create datetime index from year (iy), day of year (id), and hour (it)
    df['datetime'] = pd.to_datetime(df['iy'].astype(int).astype(str), format='%Y') + \
                            pd.to_timedelta(df['id'] - 1, unit='D') + \
                            pd.to_timedelta(df['it'], unit='h')

    # Set datetime as index
    df.set_index('datetime', inplace=True)

    # Select all records from the period of interest
    df_selection = df.loc[day:day]

    # Save to new file in the same format
    df_selection.to_csv(met_file_selection, sep=' ', index=False)

    print(f"> UMEP meteo-forcing for {day} saved as:\n{met_file_selection}")

In [ ]:
def update_solweigini(
    MODEL_SOURCE_FOLDER,
    MODEL_INI_ORIGINAL,
    MODEL_INI_UPDATED,
    MODEL_INPUT_FOLDER,
    MODEL_OUTPUT_FOLDER,
    cityInfo,
):

    import os
    import configparser

    # Step 1: Read the original config
    config = configparser.ConfigParser()
    config.optionxform = str  # preserve case
    config.read(MODEL_INI_ORIGINAL)

    # Step 2: Modify parameters in the DEFAULT section
    config['DEFAULT']['output_dir'] = MODEL_OUTPUT_FOLDER
    config['DEFAULT']['working_dir'] = MODEL_OUTPUT_FOLDER
    config['DEFAULT']['para_json_path'] = os.path.join(MODEL_SOURCE_FOLDER, "umep", "parametersforsolweig.json")
    config['DEFAULT']['filepath_dsm'] = os.path.join(MODEL_INPUT_FOLDER, "dsm.tif")
    config['DEFAULT']['filepath_cdsm'] = os.path.join(MODEL_INPUT_FOLDER, "cdsm.tif")
    config['DEFAULT']['filepath_tdsm']
    config['DEFAULT']['filepath_dem'] = os.path.join(MODEL_INPUT_FOLDER, "dem.tif")
    config['DEFAULT']['filepath_lc'] = os.path.join(MODEL_INPUT_FOLDER, "lc.tif")
    config['DEFAULT']['filepath_wh'] = os.path.join(MODEL_OUTPUT_FOLDER, "walls", "wall_hts.tif")
    config['DEFAULT']['filepath_wa'] = os.path.join(MODEL_OUTPUT_FOLDER, "walls", "wall_aspects.tif")
    config['DEFAULT']['input_svf'] = os.path.join(MODEL_OUTPUT_FOLDER, "svf", "svfs.zip")
    config['DEFAULT']['input_aniso'] = os.path.join(MODEL_OUTPUT_FOLDER, "svf", "shadowmats.npz")
    # config['DEFAULT']['poi_file']
    # config['DEFAULT']['poi_field']
    # config['DEFAULT']['input_wall']
    # config['DEFAULT']['woi_file']
    # config['DEFAULT']['woi_field']
    config['DEFAULT']['input_met'] = os.path.join(MODEL_INPUT_FOLDER, f"solweig_forcing_{cityInfo['day_of_interest']}.txt")
    # config['DEFAULT']['standalone']
    # config['DEFAULT']['onlyglobal']
    config['DEFAULT']['usevegdem']
    config['DEFAULT']['conifer_bool']= f"{cityInfo['leaf_on_trees_full_year']}"  # Leaf on trees full year
    # config['DEFAULT']['cyl']
    config['DEFAULT']['utc'] = f"{cityInfo['utc_offset']}" # Offset from UTC
    # config['DEFAULT']['useepwfile']
    # config['DEFAULT']['landcover']
    # config['DEFAULT']['demforbuild']
    # config['DEFAULT']['aniso']
    # config['DEFAULT']['wallscheme']
    # config['DEFAULT']['walltype']
    config['DEFAULT']['outputtmrt'] = '1'
    config['DEFAULT']['outputkup'] = '0'
    config['DEFAULT']['outputkdown'] = '0'
    config['DEFAULT']['outputlup'] = '0'
    config['DEFAULT']['outputldown'] = '0'
    config['DEFAULT']['outputsh'] = '0'
    config['DEFAULT']['savebuild'] = '0'
    config['DEFAULT']['outputkdiff'] = '0'
    config['DEFAULT']['outputtreeplanter'] = '0'
    config['DEFAULT']['wallnetcdf'] = '0'
    # config['DEFAULT']['date1']
    # config['DEFAULT']['date2']

    # Step 3: Save to a new location
    with open(MODEL_INI_UPDATED, 'w') as configfile:
      config.write(configfile)
    print(f"Updated configsolweig.ini saved to: {MODEL_INI_UPDATED}")

    return config

In [ ]:
def plot_walls_and_svf(filepath_wh, filepath_wa, svf_total_path, figname):

    import matplotlib.pyplot as plt
    import rasterio
    import numpy as np
    import os

    # Open the raster files
    with rasterio.open(filepath_wh) as src_wh, \
        rasterio.open(filepath_wa) as src_wa, \
        rasterio.open(svf_total_path) as src_svf:

        # Read the raster data
        wall_hts = src_wh.read(1)
        aspects = src_wa.read(1)
        svf_total = src_svf.read(1)

        # Mask 0 values in wall_hts and aspects
        wall_hts_masked = np.ma.masked_equal(wall_hts, 0)
        aspects_masked = np.ma.masked_equal(aspects, 0)

        # Create a figure with three subplots side by side
        fig, axes = plt.subplots(1, 3, figsize=(15, 7))

        # Plot wall_hts
        im1 = axes[0].imshow(wall_hts_masked, cmap='viridis')
        axes[0].set_title('Wall Heights')
        axes[0].axis('off')
        fig.colorbar(im1, ax=axes[0], orientation='horizontal', label='Height')

        # Plot aspects
        im2 = axes[1].imshow(aspects_masked, cmap='hsv') # Using hsv for aspect direction
        axes[1].set_title('Aspects')
        axes[1].axis('off')
        fig.colorbar(im2, ax=axes[1], orientation='horizontal', label='Direction (degrees)')


        # Plot svf_total
        im3 = axes[2].imshow(svf_total, cmap='inferno')
        axes[2].set_title('Sky View Factor')
        axes[2].axis('off')
        fig.colorbar(im3, ax=axes[2], orientation='horizontal', label='SVF')

        plt.tight_layout()

        plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
        print("File saved:", os.path.exists(figname))

        plt.show()

In [ ]:
# Helper functions to be used in calculate_utci
# Convert individual mrt geotif to dataarray
def _parse_datetime_from_filename(file):
    """ Helper function to read_merge_geotiff_to_dataarray"""
    import pandas as pd

    filename = file.split('/')[-1]
    parts = filename.split('_')
    year = int(parts[1])
    doy = int(parts[2])
    hour = int(parts[3][:4]) // 100  # Extract hour and convert to integer
    return pd.to_datetime(f'{year}-{doy}', format='%Y-%j') + pd.to_timedelta(hour, unit='h')

def _load_and_assign_time(filepath, epsg):
    """Helper function to read_merge_geotiff_to_dataarray"""
    import pandas as pd
    import xarray as xr

    datetime = _parse_datetime_from_filename(str(filepath))
    # Open the raster file
    ds = xr.open_dataset(filepath, engine='rasterio')
    ds = ds.rio.reproject(epsg)
    # Assign time coordinate
    ds = ds.expand_dims(time=[datetime])
    return ds

def read_merge_geotiff_to_dataarray(fileList, var_name, epsg='EPSG:4326'):
    """ Read tmrt geotifs and combine to xarray dataset in lat lon"""
    import pandas as pd
    import xarray as xr

    # Load all files with assigned times
    datasets = [_load_and_assign_time(str(f), epsg) for f in fileList]
    ds = xr.concat(datasets, dim='time')
    # Drop 'band' dimensions
    ds_squeezed = ds.squeeze('band', drop=True)
    # Rename data variable
    ds_final = ds_squeezed.rename({'band_data': var_name})
    return ds_final

import sys
import os
import contextlib

@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


def calculate_utci(config):

    with suppress_output():
        import os
        from pythermalcomfort.models import utci as calc_utci
        import glob
        import xarray as xr
        import rioxarray as rxr
        import numpy as np
        import pandas as pd
        import warnings

    import warnings
    warnings.filterwarnings("ignore")

    # List the Tmrt output files
    tmrt_tif_files = glob.glob(f"{config['DEFAULT']['output_dir']}/Tmrt_*_*_*.tif")
    tmrt_tif_files.sort()
    print(f"> Found {len(tmrt_tif_files)} Tmrt files")

    # Create dataarray
    tmrt = read_merge_geotiff_to_dataarray(tmrt_tif_files, var_name='Tmrt', epsg=rxr.open_rasterio(tmrt_tif_files[0]).rio.crs)

    # Read forcing data for selected day
    df_forcing = pd.read_csv(config['DEFAULT']['input_met'], sep=" ")

    # Broadcasting variables to fit Tmrt
    Tair_br = np.broadcast_to(np.array(df_forcing['Tair'])[:, np.newaxis, np.newaxis], tmrt['Tmrt'].shape)
    RH_br = np.broadcast_to(np.array(df_forcing['RH'])[:, np.newaxis, np.newaxis], tmrt['Tmrt'].shape)
    U_br = np.broadcast_to(np.array(df_forcing['U'])[:, np.newaxis, np.newaxis], tmrt['Tmrt'].shape)

    meteo_dict = {
        'Tmrt': tmrt['Tmrt'].data,
        'Tair': Tair_br,
        'RH': RH_br,
        'U': U_br
    }

    for var in meteo_dict.keys():
      print(f"> Ranges {var} (min | max): {meteo_dict[var].min()} | {meteo_dict[var].max()}")

    # Get UTCI
    print("> Calculating spatial UTCI ...")
    utci_results = calc_utci(
        tdb=Tair_br,
        tr=tmrt['Tmrt'].data,
        v=U_br,
        rh=RH_br,
        # return_stress_category=False,
        limit_inputs=False,
    )

    # Convert to xarray dataArray
    ds_utci = xr.Dataset({
        "utci": xr.DataArray(
            data=utci_results.utci,
            dims=["time", "y", "x"],
            coords=tmrt['Tmrt'].coords
        )
    })

    # Save to netcdf
    FN_DS_UTCI = os.path.join(config['DEFAULT']['output_dir'], "utci.nc")
    ds_utci.to_netcdf(FN_DS_UTCI)
    print(f"> UTCI dataset saved as {FN_DS_UTCI}")

    return ds_utci


In [ ]:
def plot_mean_daytime_utci(config, filepath_utci, hours_of_interest, figname):

    import os
    import rioxarray as rxr
    import numpy as np
    import xarray as xr
    import matplotlib.pyplot as plt
    import geopandas as gpd

    # Read the data
    ds_cdsm = rxr.open_rasterio(config['DEFAULT']['filepath_cdsm']).squeeze()
    ds_dem = rxr.open_rasterio(config['DEFAULT']['filepath_dem']).squeeze()
    ds_dsm = rxr.open_rasterio(config['DEFAULT']['filepath_dsm']).squeeze()
    ds_utci = xr.open_dataset(filepath_utci).squeeze()

    # Get daytime UTCI average
    ds_utci = ds_utci.sel(time=ds_utci.time.dt.hour.isin(hours_of_interest))
    ds_utci = ds_utci['utci'].mean(dim=['time'])

    # Remove DEM from DSM
    ds_dsm = ds_dsm-ds_dem

    # Mask pixels without buildings
    ds_dsm = ds_dsm.where(ds_dsm >= 2, np.nan)

    # Mask pixels without trees
    ds_cdsm = ds_cdsm.where(ds_cdsm != 0, np.nan)

    fig, ax = plt.subplots(1, 2, figsize=(16, 8), gridspec_kw={"width_ratios": [1, 1]}, tight_layout=True)

    # --- LEFT: Buildings and Trees ---
    dsm_plot = ds_dsm.plot(ax = ax[0],
        cmap='Greys',
        alpha=1,
        add_colorbar=False,
        # interpolation='none',
        # origin='upper'
    )
    cdsm_plot = ds_cdsm.plot(ax = ax[0],
        cmap='Greens',
        alpha=0.6,
        add_colorbar=False,
        # interpolation='none',
        # origin='upper'
    )
    ax[0].set_title("Buildings and Tree Height", fontsize=14)
    ax[0].axis('off')

    # --- RIGHT: UTCI values ---
    min_max_values = ds_utci.quantile([0.0, 1])
    utci_plot = ds_utci.plot(ax = ax[1],
        cmap='turbo',
        vmin=min_max_values[0].item(),
        vmax=min_max_values[1].item(),
        add_colorbar=False,
        # interpolation='none',
        # origin='upper'
    )

    # --- Add roofs on top of UTCI plot (right panel) ---
    gdf_roofs = gpd.read_file(os.path.join(os.path.dirname(config['DEFAULT']['filepath_dsm']), "roofs.geojson"))

    # (Optional: reproject if needed to match ds_utci)
    gdf_roofs = gdf_roofs.to_crs(ds_utci.rio.crs)

    gdf_roofs.plot(
        ax=ax[1],
        # column='ROOF_HEIGH',
        # cmap='Greys',
        linewidth=0,
        edgecolor='none',
        facecolor="white",
        hatch='xxxx',
        legend=False
    )


    ax[1].set_title(f"Mean daytime UTCI", fontsize=14)
    ax[1].axis('off')

    # --- COLORBARS BELOW ---

    # Create a horizontal axis space below the figure for the colorbars
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes

    # 1. Left plot: CDSM and DSM colorbars side by side
    inset_ax1 = inset_axes(ax[0], width="45%", height="3%", loc='lower left', bbox_to_anchor=(0.0, -0.05, 1, 1), bbox_transform=ax[0].transAxes, borderpad=0)
    inset_ax2 = inset_axes(ax[0], width="45%", height="3%", loc='lower left', bbox_to_anchor=(0.5, -0.05, 1, 1), bbox_transform=ax[0].transAxes, borderpad=0)

    cb1 = fig.colorbar(dsm_plot, cax=inset_ax1, orientation='horizontal')
    cb1.set_label("Building Height [m]", fontsize=12)
    cb1.ax.tick_params(labelsize=11)

    cb2 = fig.colorbar(cdsm_plot, cax=inset_ax2, orientation='horizontal')
    cb2.set_label("Tree Height [m]", fontsize=12)
    cb2.ax.tick_params(labelsize=11)

    # 2. Right plot: UTCI colorbar full width
    inset_ax3 = inset_axes(ax[1], width="90%", height="3%", loc='lower center', bbox_to_anchor=(0.0, -0.05, 1, 1), bbox_transform=ax[1].transAxes, borderpad=0)

    cb3 = fig.colorbar(utci_plot, cax=inset_ax3, orientation='horizontal')
    cb3.set_label(f'UTCI (°C)', fontsize=12)
    cb3.ax.tick_params(labelsize=11)

    for a in ax:
        a.set_aspect('equal')  # or 'auto' if the data is rectangular

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    # Final layout
    plt.show()


In [ ]:
def check_position_pois(pois):

    import folium

    # Create a base map
    m = folium.Map(
        location=[pois[0][0], pois[0][1]],
        zoom_start=17,
        tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr='Google',
        name='Google Satellite',
        width='70%',
        height='800px'
    )

    # Add markers directly to the map
    for i, poi in enumerate(pois):
        folium.Marker(
            location=[poi[0], poi[1]],
            popup=f'Point {i+1}',
        ).add_to(m)

    return m

In [ ]:
def plot_poi_timeseries_utci(config, filepath_utci, pois, figname):

    import os
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import xarray as xr
    import rioxarray as rxr
    from pyproj import Transformer

    # Load the UTCI dataset
    ds_utci = xr.open_dataset(filepath_utci).squeeze()

    # Convert POI coordinates to the CRS of the UTCI dataset
    utci_crs = ds_utci.rio.crs
    transformer = Transformer.from_crs("EPSG:4326", utci_crs, always_xy=True)
    pois_reprojected = []
    for lat, lon in pois: # Note: Transformer expects (longitude, latitude) for EPSG:4326
        x, y = transformer.transform(lon, lat)
        pois_reprojected.append((y, x))

    # Extract time series data for each POI
    poi_utci_timeseries = {}
    for i, (y, x) in enumerate(pois_reprojected):
      utci_series = ds_utci['utci'].sel(x=x, y=y, method='nearest')
      # Convert to pandas Series for easier plotting
      poi_utci_timeseries[f'POI {i+1}'] = utci_series.to_series()

    # --- Define UTCI categories and colors ---
    utci_bands = [
        # (-500, -40, '#6a0dad', 'extreme cold stress'),
        # (-40, -27, '#0000cc', 'very strong cold stress'),
        # (-27, -13, '#0099ff', 'strong cold stress'),
        # (-13, 0, '#33ccff', 'moderate cold stress'),
        # (0, 9, '#66ffff', 'slight cold stress'),
        (9, 26, '#66ff66', 'no thermal stress'),
        (26, 32, '#ffcc00', 'moderate heat stress'),
        (32, 38, '#ff9900', 'strong heat stress'),
        (38, 46, '#ff3300', 'very strong heat stress'),
        (46, 60, '#cc0000', 'extreme heat stress')
    ]

    # Plot the time series
    fig, ax = plt.subplots(figsize=(10, 6))

    # --- Add background bands and category labels ---
    for lower, upper, color, label in utci_bands:
        ax.axhspan(lower, upper, facecolor=color, alpha=0.3, zorder=0)
        mid = ((lower + upper) / 2) * 1.01
        ax.text(
            x=0.05, y=mid, s=label,
            va='center', ha='left', fontsize=9, color='black',
            transform=ax.get_yaxis_transform(),
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.3')
        )

    for poi_name, ts in poi_utci_timeseries.items():
        plt.plot(ts.index.hour, ts.values, label=poi_name, marker='o', linestyle='-')

    # Format the plot
    plt.title(f'UTCI Timeseries for points of interest')
    plt.xlabel('Hour of Day')
    plt.ylabel('UTCI (°C)')
    plt.xticks(np.arange(0, 24, 1)) # Ensure x-axis shows all 24 hours
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.ylim([15, 60])
    plt.legend()

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()

In [ ]:
def plot_mean_daytime_utci_difference_scenarios(config, filepath_utci_base, filepath_utci_scen, hours_of_interest, figname):

    import os
    import rioxarray as rxr
    import numpy as np
    import xarray as xr
    import matplotlib.pyplot as plt
    import geopandas as gpd

    # Read the data
    ds_cdsm = rxr.open_rasterio(config['DEFAULT']['filepath_cdsm']).squeeze()
    ds_dem = rxr.open_rasterio(config['DEFAULT']['filepath_dem']).squeeze()
    ds_dsm = rxr.open_rasterio(config['DEFAULT']['filepath_dsm']).squeeze()
    ds_utci_base = xr.open_dataset(filepath_utci_base).squeeze()
    ds_utci_scen = xr.open_dataset(filepath_utci_scen).squeeze()

    # Get daytime UTCI average
    ds_utci_base = ds_utci_base.sel(time=ds_utci_base.time.dt.hour.isin(hours_of_interest))
    ds_utci_base = ds_utci_base['utci'].mean(dim=['time'])

    ds_utci_scen = ds_utci_scen.sel(time=ds_utci_scen.time.dt.hour.isin(hours_of_interest))
    ds_utci_scen = ds_utci_scen['utci'].mean(dim=['time'])

    # Their difference
    ds_utci_diff = ds_utci_scen - ds_utci_base

    # Vmin and max for colorrange?
    base_min = ds_utci_base.min().item()
    base_max = ds_utci_base.max().item()
    scen_min = ds_utci_scen.min().item()
    scen_max = ds_utci_scen.max().item()
    vmin = min(base_min, scen_min)
    vmax = max(base_max, scen_max)

    # diff_min = ds_utci_diff.min().item()
    # diff_max = ds_utci_diff.max().item()
    # vmin = min(-diff_max, diff_min)
    # vmax = max(vmax, diff_max)

    fig, ax = plt.subplots(1, 3, figsize=(16, 6), tight_layout=True) # gridspec_kw={"width_ratios": [1, 1, 1]},

    base_name = os.path.dirname(filepath_utci_base).split("/")[-1]
    base_plot = ds_utci_base.plot(ax = ax[0],
        cmap='turbo',
        alpha=1,
        add_colorbar=True,
        cbar_kwargs={"location": "bottom"},
        vmin=vmin,
        vmax=vmax,
    )
    ax[0].set_title(f"Scenario: {base_name}", fontsize=14)
    ax[0].axis('off')

    scen_name = os.path.dirname(filepath_utci_scen).split("/")[-1]
    scen_plot = ds_utci_scen.plot(ax = ax[1],
        cmap='turbo',
        alpha=1,
        add_colorbar=True,
        cbar_kwargs={"location": "bottom"},
        vmin=vmin,
        vmax=vmax,
    )
    ax[1].set_title(f"Scenario: {scen_name}", fontsize=14)
    ax[1].axis('off')

    utci_diff_plot = (ds_utci_diff).plot(ax = ax[2],
        cmap='coolwarm',
        # vmin=min_max_values[0].item(),
        # vmax=min_max_values[1].item(),
        add_colorbar=True,
        cbar_kwargs={"location": "bottom"},
    )
    ax[2].set_title(f"Difference: {scen_name} - {base_name}", fontsize=14)
    ax[2].axis('off')

    for a in ax:
        a.set_aspect('equal')  # or 'auto' if the data is rectangular

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    # Final layout
    plt.show()

In [ ]:
def plot_poi_timeseries_utci_scenarios(config, filepath_utci_base, filepath_utci_scen, pois, figname):

    import os
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import xarray as xr
    import rioxarray as rxr
    from pyproj import Transformer

    # Load the UTCI dataset
    ds_utci_base = xr.open_dataset(filepath_utci_base).squeeze()
    ds_utci_scen = xr.open_dataset(filepath_utci_scen).squeeze()

    base_name = os.path.dirname(filepath_utci_base).split("/")[-1]
    scen_name = os.path.dirname(filepath_utci_scen).split("/")[-1]

    # Convert POI coordinates to the CRS of the UTCI dataset
    utci_crs = ds_utci_base.rio.crs
    transformer = Transformer.from_crs("EPSG:4326", utci_crs, always_xy=True)
    pois_reprojected = []
    for lat, lon in pois: # Note: Transformer expects (longitude, latitude) for EPSG:4326
        x, y = transformer.transform(lon, lat)
        pois_reprojected.append((y, x))

    # Extract time series data for each POI and each scenario
    poi_utci_base_timeseries = {}
    for i, (y, x) in enumerate(pois_reprojected):
      utci_base_series = ds_utci_base['utci'].sel(x=x, y=y, method='nearest')
      # Convert to pandas Series for easier plotting
      poi_utci_base_timeseries[f'POI {i+1}'] = utci_base_series.to_series()

    poi_utci_scen_timeseries = {}
    for i, (y, x) in enumerate(pois_reprojected):
      utci_scen_series = ds_utci_scen['utci'].sel(x=x, y=y, method='nearest')
      # Convert to pandas Series for easier plotting
      poi_utci_scen_timeseries[f'POI {i+1}'] = utci_scen_series.to_series()

    # --- Define UTCI categories and colors ---
    utci_bands = [
        # (-500, -40, '#6a0dad', 'extreme cold stress'),
        # (-40, -27, '#0000cc', 'very strong cold stress'),
        # (-27, -13, '#0099ff', 'strong cold stress'),
        # (-13, 0, '#33ccff', 'moderate cold stress'),
        # (0, 9, '#66ffff', 'slight cold stress'),
        (9, 26, '#66ff66', 'no thermal stress'),
        (26, 32, '#ffcc00', 'moderate heat stress'),
        (32, 38, '#ff9900', 'strong heat stress'),
        (38, 46, '#ff3300', 'very strong heat stress'),
        (46, 60, '#cc0000', 'extreme heat stress')
    ]

    # Plot the time series
    fig, ax = plt.subplots(figsize=(10, 6))

    # --- Add background bands and category labels ---
    for lower, upper, color, label in utci_bands:
        ax.axhspan(lower, upper, facecolor=color, alpha=0.3, zorder=0)
        mid = ((lower + upper) / 2) * 1.01
        ax.text(
            x=0.05, y=mid, s=label,
            va='center', ha='left', fontsize=9, color='black',
            transform=ax.get_yaxis_transform(),
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.3')
        )

    poi_colors = {
        'POI 1': '#e41a1c',
        'POI 2': '#377eb8',
        'POI 3': '#4daf4a',
        'POI 4': '#984ea3',
        'POI 5': '#ff7f00',
    }

    for poi_name, ts in poi_utci_base_timeseries.items():
        plt.plot(ts.index.hour, ts.values, label=poi_name, marker='o', linestyle='-', color=poi_colors[poi_name])
    for poi_name, ts in poi_utci_scen_timeseries.items():
        plt.plot(ts.index.hour, ts.values, label=poi_name, marker='*', linestyle=':', color=poi_colors[poi_name])

    # Format the plot
    plt.title(f'UTCI Timeseries for points of interest for scenarios {base_name} (o) and {scen_name} (*)')
    plt.xlabel('Hour of Day')
    plt.ylabel('UTCI (°C)')
    plt.xticks(np.arange(0, 24, 1)) # Ensure x-axis shows all 24 hours
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.ylim([15, 60])
    plt.legend()

    plt.tight_layout()

    plt.savefig(figname, dpi=300, bbox_inches='tight', pad_inches=0.15)
    print("File saved:", os.path.exists(figname))

    plt.show()